In [1]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 2.9 MB 7.5 MB/s 
     |████████████████████████████████| 895 kB 50.7 MB/s 
     |████████████████████████████████| 3.3 MB 43.2 MB/s 
     |████████████████████████████████| 61 kB 498 kB/s 
     |████████████████████████████████| 596 kB 53.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 298 kB 9.3 MB/s 
     |████████████████████████████████| 1.1 MB 44.5 MB/s 
     |████████████████████████████████| 132 kB 64.6 MB/s 
     |████████████████████████████████| 243 kB 61.6 MB/s 
     |████████████████████████████████| 271 kB 69.6 MB/s 
     |████████████████████████████████| 160 kB 65.6 MB/s 
     |████████████████████████████████| 192 kB 64.8 MB/s 


In [2]:
from datasets import load_dataset, DatasetDict

dataset = load_dataset("yxchar/sciie-tlm")
dataset.num_rows

Using custom data configuration yxchar___sciie-tlm-a32f1f2c4e9b5c0d


  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/663k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/199k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/yxchar___sciie-tlm-a32f1f2c4e9b5c0d/0.0.0/bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

{'test': 974, 'train': 3219, 'validation': 455}

In [3]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], max_length=512, truncation=True, padding="max_length")

# Encode the input data
dataset = dataset.map(encode_batch, batched=True)
# The transformers model expects the target class column to be named "labels"
dataset.rename_column_("label", "labels")
# Transform to pytorch tensors and only output the required columns
dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: FutureWarning: rename_column_ is deprecated and will be removed in the next major version of datasets. Use DatasetDict.rename_column instead.
  if sys.path[0] == '':


In [4]:
from transformers import RobertaConfig, RobertaModelWithHeads

config = RobertaConfig.from_pretrained(
    "roberta-base",
    num_labels=7,
)
model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=config,
)

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModelWithHeads: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModelWithHeads from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModelWithHeads from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModelWithHeads were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infere

In [5]:
# Add a new adapter
model.add_adapter("sciie")
# Add a matching classification head
model.add_classification_head(
    "sciie",
    num_labels=7,
    # id2label={ 0: "👎", 1: "👍"}
  )

# Activate the adapter
model.train_adapter("sciie")

In [6]:
import numpy as np
from transformers import TrainingArguments, AdapterTrainer, EvalPrediction

training_args = TrainingArguments(
    learning_rate=5e-5,
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
)

def compute_accuracy(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis=1)
  return {"acc": (preds == p.label_ids).mean()}

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_accuracy,
)

In [7]:
trainer.train()

***** Running training *****
  Num examples = 3219
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3030


Step,Training Loss
200,1.582400
400,1.353100
600,1.087100
800,0.909000
1000,0.770800
1200,0.654500
1400,0.627300
1600,0.559300
1800,0.509300
2000,0.491000


Saving model checkpoint to ./training_output/checkpoint-500
Configuration saved in ./training_output/checkpoint-500/sciie/adapter_config.json
Module weights saved in ./training_output/checkpoint-500/sciie/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-500/sciie/head_config.json
Module weights saved in ./training_output/checkpoint-500/sciie/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/sciie/head_config.json
Module weights saved in ./training_output/checkpoint-500/sciie/pytorch_model_head.bin
Configuration saved in ./training_output/checkpoint-500/sciie/head_config.json
Module weights saved in ./training_output/checkpoint-500/sciie/pytorch_model_head.bin
Saving model checkpoint to ./training_output/checkpoint-1000
Configuration saved in ./training_output/checkpoint-1000/sciie/adapter_config.json
Module weights saved in ./training_output/checkpoint-1000/sciie/pytorch_adapter.bin
Configuration saved in ./training_output/checkpoint-

TrainOutput(global_step=3030, training_loss=0.7094132942728477, metrics={'train_runtime': 1947.7147, 'train_samples_per_second': 24.791, 'train_steps_per_second': 1.556, 'total_flos': 1.29251777497344e+16, 'train_loss': 0.7094132942728477, 'epoch': 15.0})

In [8]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 455
  Batch size = 16


{'epoch': 15.0,
 'eval_acc': 0.8373626373626374,
 'eval_loss': 0.4363047778606415,
 'eval_runtime': 9.0225,
 'eval_samples_per_second': 50.43,
 'eval_steps_per_second': 3.214}

In [9]:
from transformers import TextClassificationPipeline

classifier = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=training_args.device.index)

classifier("This is awesome!")

[{'label': 'LABEL_3', 'score': 0.3081027865409851}]

In [10]:
model.save_adapter("./final_adapter", "sciie")

!ls -lh final_adapter

Configuration saved in ./final_adapter/adapter_config.json
Module weights saved in ./final_adapter/pytorch_adapter.bin
Configuration saved in ./final_adapter/head_config.json
Module weights saved in ./final_adapter/pytorch_model_head.bin


total 5.8M
-rw-r--r-- 1 root root  571 Dec 15 22:53 adapter_config.json
-rw-r--r-- 1 root root  477 Dec 15 22:53 head_config.json
-rw-r--r-- 1 root root 3.5M Dec 15 22:53 pytorch_adapter.bin
-rw-r--r-- 1 root root 2.3M Dec 15 22:53 pytorch_model_head.bin


In [11]:
from google.colab import files
!zip -r /content/file.zip /content
files.download("/content/file.zip")

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/config_sentinel (stored 0%)
  adding: content/.config/.last_survey_prompt.yaml (stored 0%)
  adding: content/.config/.last_update_check.json (deflated 24%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2021.12.03/ (stored 0%)
  adding: content/.config/logs/2021.12.03/14.32.30.027140.log (deflated 91%)
  adding: content/.config/logs/2021.12.03/14.33.37.701606.log (deflated 53%)
  adding: content/.config/logs/2021.12.03/14.33.36.903459.log (deflated 54%)
  adding: content/.config/logs/2021.12.03/14.33.16.964195.log (deflated 54%)
  adding: content/.config/logs/2021.12.03/14.33.09.955489.log (deflated 86%)
  adding: content/.config/logs/2021.12.03/14.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
pred, label, metrics = trainer.predict(dataset["test"])
pred_label = np.argmax(pred, axis = 1)

from sklearn.metrics import classification_report 
print(classification_report(label, pred_label))

***** Running Prediction *****
  Num examples = 974
  Batch size = 16


              precision    recall  f1-score   support

           0       0.88      0.93      0.91       123
           1       0.55      0.56      0.55        59
           2       0.82      0.90      0.86        67
           3       0.90      0.90      0.90       533
           4       0.53      0.37      0.43        63
           5       0.67      0.82      0.74        38
           6       0.80      0.76      0.78        91

    accuracy                           0.83       974
   macro avg       0.74      0.75      0.74       974
weighted avg       0.83      0.83      0.83       974

